# Algebra of drawing functions

In this chapter we will close the circle of functions, by showing that functions can also be used to represent complex data structures and their aggregation. The naïve interpretation of functions imposes a somewhat false dichotomy between state and actions, where functions are merely relegated to containers of actions.

A broader interpretation of functions would begin by realising that state itself can be easily modeled as function. For example, there is no actual difference between a number, such as `3`, and a function that always returns that number, such as `three := () => 3`. The fact that the function definition will first require a call does not change the fact that it merely acts as a container for `3`, and as such the two definitions are equivalent. Moreover, given any piece of state `C`, we could encapsulate it in a function `() => C` which does nothing but returning its value. This means that for every value we could possibly conceive, there exists a function which sole purpose is to return it. This means that `State <= Functions`. The opposite is not true. Simplifying broadly, we can easily imagine that even if we managed to encode functions cleverly in the form of variables, we would still at some point need functions in order to evaluate such an encoding, or _call the function_ as it were. 

Thus, we can conclude that functions are the fundamental unit of computation, and even more so the only construct we should really be focusing on. Indeed, even concepts such as object-orientation, inheritance, and much more are trivial to encode and represent only in terms of functions and their composition. We will see more of this in the coming units, but in the rest of this chapter we will show how to build a simple (but not trivial) data structure representing pictures only by means of functions.

## A general drawing function

Let us go back to the simplest form of drawing functions: just two nested loops, and at each iteration we simply check whether or not the current pixel should be "lit" (an asterisk) or not (a space). The check will need to know the total size of the figure being drawn (width and height) plus the coordinates of the current pixel (x and y, or j and i).

Depending on the check we perform at each pixel, we could draw completely different figures, but the general shape of the drawing function will not change. For this reason we can build the general concept, without any mention of the specific figure being drawn, as a higher order function which accepts the check per pixel as a parameter:

In [1]:
def draw(n,m,p):
    s = ""
    i = 0
    while i < n:
        j = 0
        while j < m:
            if p(n,m,i,j):
                s = s + "*"
            else:
                s = s + " "
            j = j + 1
        s = s + "\n"
        i = i + 1
    return s

The `draw` function accepts as arguments the size of the figure to draw, plus a function `p` (stands for "picture"), which determines for each pixel whether or not it will be drawn.

This function accepts as arguments the size of the picture to draw, which we will usually call `n` and `m` (height and width respectively), and of course the coordinates of the current pixel, which we will usually call `i` and `j` (row and column respectively). By checking the coordinates against each other and against the size of the picture, the function determines relevant properties of the pixel (such as being on the left side, or `j == 0`) and concludes whether or not the pixel must be lit.

For example, by using a function which always returns `True`, we get a full square; by using a function which always returns `False`, we get an "invisible" square; if we use a function which checks whether or not `j <= i`, we only keep the lower right triangle on; and so on...

In [18]:
print(draw(3,3,lambda n,m,i,j: True))
print(draw(3,3,lambda n,m,i,j: False))
print(draw(3,3,lambda n,m,i,j: j <= i))

***
***
***

   
   
   

*  
** 
***



At this point we could observe a powerful fact: **any function from four integers (width, height, row index, column index) into a boolean (pixel on/off) is a figure**. There is no function which corresponds to no figure, since every such function can be drawn. Perhaps some of these functions do not produce any _interesting_ figure, but it will be possible to draw it nonetheless.

We need no complex data structure to define figures, as we might expect with an object-oriented background where an inheritance hierarchy of classes is used to define a taxonomy of figures: functions are just enough.

### Combinators

Let us now consider a hollow square. We can easily identify and draw the borders of the square, which are clearly the components of the picture.

Specifically, the borders are characterized by the following checks: `i == 0`, `j == 0`, `i == n-1`, or `j == n-1`. We can easily fill in the gaps and define the lambdas as follows:

In [19]:
print(draw(3,3,lambda n,m,i,j: i == 0))
print(draw(3,3,lambda n,m,i,j: j == 0))
print(draw(3,3,lambda n,m,i,j: i == n-1))
print(draw(3,3,lambda n,m,i,j: j == m-1))

***
   
   

*  
*  
*  

   
   
***

  *
  *
  *



Of course now we need a way to compose these borders together in a single figure. We are looking for a way to compose two figures into a figure which is their union. Let us rephrase this: we are looking for a way to compose two functions into a function which itself returns `True` when any of the two original functions would have returned `True`.

Let us begin by doing this explicitly and a bit too verbosely:

In [2]:
upper_line = lambda n,m,i,j: i == 0
left_line = lambda n,m,i,j: j == 0
lower_line = lambda n,m,i,j: i == n-1
right_line = lambda n,m,i,j: j == m-1

hollow_square = lambda n,m,i,j: upper_line(n,m,i,j) or lower_line(n,m,i,j) or left_line(n,m,i,j) or right_line(n,m,i,j)
print(draw(3,3,hollow_square))

***
* *
***



Suppose now that we wanted to just draw the upper and lower rows together. This would mean we would have to define another function, and either figure out a reasonable name for it or _curry_ a cumbersome lambda around:

In [3]:
upper_and_lower_rows = lambda n,m,i,j: upper_line(n,m,i,j) or lower_line(n,m,i,j)
print(draw(3,3,upper_and_lower_rows))

***
   
***



The unpleasant verbosity comes from the repetition of `n,m,i,j`: this tuple of four integers is repeated _three times!_ in a single row.

We can get rid of this verbosity by observing that we do not really wish to think about the parameters, and our target code should look more like `union(upper_line, lower_line)`, which clearly reflects our thoughts.

Such pattern could be defined explicitly as a higher order function which takes the two figures as parameters, invokes both of them, and combines the results with an `or`:

In [4]:
def union(p,q):
    return lambda n,m,i,j: p(n,m,i,j) or q(n,m,i,j)
print(draw(3,3, union(upper_line, lower_line)))

***
   
***



We call higher order functions such as `union` _combinators_, since they _combine_ arbitrary complex data structures together. Thanks to combinators, it is usually possible to very quickly assemble existing structures into new forms, and all the while retaining elegant and expressive code:  

In [22]:
print(draw(3,3,union(upper_line, left_line)))
print(draw(6,6,union(lambda n,m,i,j: n-i-1 <= j, lambda n,m,i,j: j <= i)))

***
*  
*  

*    *
**  **
******
******
******
******



Moreover, good combinators can nest arbitrarily deep. Indeed, the result of `union` is itself a figure, therefore we can just pass it around as a figure, even as a parameter to another `union` itself. The ability to arbitrarily nest is very important, as it guarantees that there is no artificial upper bound to our ability to express deeply articulated concepts. 

Indeed, how could we possibly guarantee that no interesting figure exists which requires more than a fixed level of nested combinations?

Such a nested combination would perform the union of the horizontal borders and the vertical borders, in order to produce a succint and semantically recognizable definition of hollow squares:

In [23]:
print(draw(5,5, union(union(upper_line, lower_line), union(left_line, right_line))))

*****
*   *
*   *
*   *
*****



Of course `union` is not the only combinator possible. Instead of unstructuredly exploring combinators by following our experience and intuition, we could simply observe that since the figure functions all return boolean values, we can merely perform all operations that are allowed on booleans on their return types, therefore leading us to multiple combinators without having to think much. We have already seen `or`, which led us to the `union`. We call this the _lifting_ of a boolean operator from the (concreter) domain of booleans into the (abstracter) domain of figure functions. 

<div class="alert alert-block alert-info">
Lifting here refers to moving higher up, and of course assumes the intuition that abstraction grows upwards. We could just turn everything upside down and image that abstraction lies at the bottom, but then we would need to say _digging_ instead of _lifting_, which is now widely accepted as a common idiom. Nevertheless, it is all quite arbitrary.
</div>

By further generalizing the `union`, we could notice that all binary combinators (binary in the sense of accepting two figures `p` and `q` as input) might look like:

```
def COMBINATOR(p,q):
    return lambda n,m,i,j: p(n,m,i,j) BOOLEAN_OPERATION q(n,m,i,j)
```

For example, we could perform the `and` of the result of the figure parameter(s), thereby obtaining the intersection of figures. Similarly, we could try the same strategy on unary operators such as `not`, leading us to the inversion of figures:

In [11]:
def pic_and(p,q):
    return lambda n,m,i,j: p(n,m,i,j) and q(n,m,i,j)
print(draw(3,3,pic_and(upper_line, left_line)))

def pic_or(p,q):
    return union(p,q)
print(draw(3,3,pic_or(upper_line, left_line)))

def pic_not(p):
    return lambda n,m,i,j: not p(n,m,i,j)
print(draw(3,3,pic_not(left_line)))

*  
   
   

***
*  
*  

 **
 **
 **



Manipulating multiple return values with boolean operators is one strategy for combining and transforming figures. We can of course also perform some transformation of the parameters, for example with offsets, scales, or other transformations. Here we only show the flipping transformation which horizontally or vertically turns a picture around its middle axis.

This is simply done by, for example, invoking the actual figure function with `n-1` instead of `0`, `n-2` instead of `1`, etc.

In [12]:
def flip_hor(p):
    return lambda n,m,i,j: p(n,m,i,m-1-j)
def flip_ver(p):
    return lambda n,m,i,j: p(n,m,n-1-i,j)
upper_right_triangle = lambda n,m,i,j: i <= j
upper_left_triangle  = flip_hor(upper_right_triangle)
lower_right_triangle = flip_ver(upper_right_triangle)
lower_left_triangle  = flip_ver(upper_left_triangle)
print(draw(3,3,upper_left_triangle))
print(draw(3,3,upper_right_triangle))
print(draw(3,3,lower_left_triangle))
print(draw(3,3,lower_right_triangle))

***
** 
*  

***
 **
  *

*  
** 
***

  *
 **
***



Complex shapes which required quite a lot of thought, such as the pyramid, can now simply be decomposed into simpler shapes such as two lower triangles. Flipping helps us also in the building of variations of the same figure, such as a rotated pyramid, which would have required radically different code (with respect to the horizontal pyramid) to build. 

<div class="alert alert-block alert-info">
This mismatch in the code that implements similar figures in very different ways (such as horizontal and vertical pyramids) can become a very serious issue: it severly limits our ability to quickly experiment with related concepts, and therefore limits our ability to build flexible and robust software. 
</div>

In [13]:
upper_pyramid = pic_and(upper_left_triangle, upper_right_triangle)
lower_pyramid = pic_and(lower_left_triangle, lower_right_triangle)
left_pyramid = pic_and(lower_left_triangle, upper_left_triangle)
right_pyramid = pic_and(lower_right_triangle, upper_right_triangle)
print(draw(5,5,lower_pyramid))
print(draw(5,5,left_pyramid))


butterfly = pic_or(left_pyramid, right_pyramid)
print(draw(5,5,butterfly))

     
     
  *  
 *** 
*****

*    
**   
***  
**   
*    

*   *
** **
*****
** **
*   *



We can even put useful formulae to the test in order to create new shapes. A good example of this could be using Pythagora's formula in order to draw circles:

In [14]:
circle = lambda n,m,i,j: (i-n//2)*(i-n//2)+(j-n//2)*(j-n//2) <= n*n//4
print(draw(5,5,circle))

 *** 
*****
*****
*****
 *** 



By combining circles and other figures we can even draw some recognizable figures quite easily:

In [16]:
eye = lambda n,m,i,j: i == n//4 and j == m//2
mouth = right_pyramid
print(draw(15,15,pic_and(circle, pic_not(pic_or(eye, mouth)))))

     *****     
   *********   
  **********   
 ****** ***    
 *********     
*********      
********       
*******        
********       
*********      
 *********     
 **********    
  **********   
   *********   
     *****     

